<a href="https://colab.research.google.com/github/mjag7682/CS9-1-NLP-for-Twitter-Data-for-predicting-stocks/blob/master/PreprocessingFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install contractions
!pip install emoji
!pip install ekphrasis

import pandas as pd
import re
import emoji
import nltk
import contractions
import torch

nltk.download('punkt')
nltk.download('stopwords')
# from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer 
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize 
from nltk.tokenize.treebank import TreebankWordDetokenizer
import requests 
from pprint import pprint
import numpy as np
from nltk.stem import PorterStemmer

import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras
# import bert
# from bert import run_classifier
# from bert import optimization
# from bert import tokenization
import os
import pprint
import json
import random
import string
import sys
from ekphrasis.classes.segmenter import Segmenter
import itertools


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_comb = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Capstone/Labelled (Percentage change)/AAPL_label2.1.csv')
combine_ds = df_comb.sample(frac=1)
combine_ds.head()

,message,label
563747,$AAPL sold my 250 April puts! BAM,-1.0
378786,Stocks from the SMB Scanner: $FB $EXPE $FEYE $...,-1.0
295238,$AAPL wish apple would just buyback to 125 to ...,0.0
110904,@zspecs @SlopeOfHope @JRNavarro75 Options can ...,1.0
1117553,$SPY $AAPL \n\nJesús Christ. That was some day!,0.0


In [5]:
combine_ds['message'] = combine_ds['message'].str.lower()
message = combine_ds['message'].tolist()

print(message[:10])


['$aapl sold my 250 april puts! bam', 'stocks from the smb scanner: $fb $expe $feye $blue $nvo $uso $aapl $expe', '$aapl wish apple would just buyback to 125 to get over the 120/700 mental barrier', '@zspecs @slopeofhope @jrnavarro75 options can also be used to reduce risk, as in these examples: http://stks.co/gne2 $aapl', '$spy $aapl \n\njesús christ. that was some day!', '$aapl forging ahead with an upward range every day...whether closing at new highs or just small gains..very bullish signal over all.', '$aapl trump supporters can&#39;t afford apple products anyway so this boycott isn&#39;t going to affect sales. rather, it will boost sales and attn', 'today&#39;s iphone event will be a test of trust for apple $aapl http://goo.gl/fpwxke', 'it&#39;s pretty clear that supply &gt;&gt; demand out there right now.. $spy $dia $qqq $aapl $nflx $nvda $pypl etc', 'the industry average profit margin is 1.64%. $aapl outperforms 97% of its industry peers. https://www.chartmill.com/stock/quote/a

In [8]:
def remove_stopwords(msg):
    filtered_sentence = [w for w in msg_tokens if not w in stop_words]
    return filtered_sentence

def remove_punctuation_re(x):
    x = ' '.join(re.sub("https?://\S+","",x).split())     #Removing URLs

    x = ' '.join(re.sub("^@\S+|\s@\S+","",x).split())     #Removing Mentions

    # x = ' '.join(re.sub(r'[^$\w\s]'," ",x).split())
    x = ' '.join(re.sub(r'[^\w\s]'," ",x).split())        #Removes Hashtags

    x = ' '.join(re.sub(r'_'," ",x).split())              #Removing _ from emojis text

    return x

# replace repeating letter
def rpt_replace(match):
    # print(match.group(1))
    return match.group(1)+match.group(1)

# substitute original word with replaced word, if any
def processRepeatings(data):
    re_t= re.sub(message_rpt, rpt_replace, data )
    # print(re_t)
    return re_t

In [9]:
stop_words = sw.words("english")
tweet_tokenizer = TweetTokenizer()
detokenizer = TreebankWordDetokenizer()
message_p = []

# for repeating characters in words
message_rpt = re.compile(r"(.)\1{2,}", re.IGNORECASE)

# segmenter using the word statistics from Twitter
seg_tw = Segmenter(corpus="twitter")

for msg in message:
    # remove emojis
    msg = emoji.demojize(msg)

    # fix contractions
    msg = contractions.fix(msg)

    # remove punctuations
    msg = remove_punctuation_re(msg) 
    message_p.append(msg)
    #tokenize
    msg_tokens = tweet_tokenizer.tokenize(msg)

    #For Hashtags elongated words using Word segmenter
    message_seg = []
    for w in msg_tokens:
      message_seg.append(seg_tw.segment(w))

    # remove stopwords
    msg = remove_stopwords(message_seg)

    if 'rt' in msg:
      # remove retweets
      message_p.append('-1')
    else: 
      # detokenize
      msg = detokenizer.detokenize(msg)

      # removing repeating words like hurrrryyyyyy-- worrks on tokenized list
      msg = processRepeatings(msg)

      message_p.append(msg)


Reading twitter - 1grams ...
Reading twitter - 2grams ...


In [12]:
message_p[:10]

['aapl sold my 250 april puts bam',
 'aapl sold 250 april puts bam',
 'stocks from the smb scanner fb expe feye blue nvo uso aapl expe',
 'stocks smb scanner fb expe feye blue nvo uso aapl expe',
 'aapl wish apple would just buyback to 125 to get over the 120 700 mental barrier',
 'aapl wish apple would buyback 125 get 120 700 mental barrier',
 'options can also be used to reduce risk as in these examples aapl',
 'options also used reduce risk examples aapl',
 'spy aapl jesús christ that was some day',
 'spy aapl jesús christ day']